In [1]:
# Install dependencies
import subprocess
import sys

try:
    subprocess.run(
        ["uv", "pip", "install", "-r", "requirements.txt"],
        capture_output=True,
        text=True,
        check=True
    )
    print("Dependencies installed successfully")
except (subprocess.CalledProcessError, FileNotFoundError):
    # Fallback to pip if uv is not available
    subprocess.run(
        [sys.executable, "-m", "pip", "install", "-r", "requirements.txt"],
        check=True
    )
    print("Dependencies installed successfully")

Dependencies installed successfully


In [7]:
# Import dependencies
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool
import json
import requests

In [8]:
# Environment variables
import os
from pathlib import Path

# Load from .env file if it exists
env_file = Path(".env")
if env_file.exists():
    with open(env_file) as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                os.environ[key] = value

# RunPod Pod endpoint (get from: https://www.runpod.io/console/pods)
# Deploy a pod with vLLM template and GPT-OSS 20B model
RUNPOD_ENDPOINT_URL = os.getenv("RUNPOD_ENDPOINT_URL", "YOUR_RUNPOD_ENDPOINT_HERE")
RUNPOD_API_KEY = os.getenv("RUNPOD_API_KEY", "YOUR_RUNPOD_API_KEY_HERE")

# Firecrawl API (get from: https://www.firecrawl.dev/)
FIRECRAWL_API_KEY = os.getenv("FIRECRAWL_API_KEY", "YOUR_FIRECRAWL_API_KEY_HERE")

print(f"Firecrawl API Key: {'Set' if FIRECRAWL_API_KEY != 'YOUR_FIRECRAWL_API_KEY_HERE' else 'Not set'}")
print(f"RunPod Endpoint: {'Set' if RUNPOD_ENDPOINT_URL != 'YOUR_RUNPOD_ENDPOINT_HERE' else 'Not set'}")

Firecrawl API Key: Set
RunPod Endpoint: Set


In [9]:
# Connect to Firecrawl MCP Server using LangChain MCP Adapters
from langchain_mcp_adapters.client import MultiServerMCPClient

# Configure Firecrawl MCP server
mcp_client = MultiServerMCPClient({
    "firecrawl": {
        "transport": "stdio",
        "command": "npx",
        "args": ["-y", "firecrawl-mcp"]
    }
})

# Get tools from MCP server
# Note: Jupyter notebooks have a running event loop, so we use await directly
firecrawl_tools = await mcp_client.get_tools()

print(f"Connected to Firecrawl MCP server")
print(f"Available tools: {[tool.name for tool in firecrawl_tools]}")

Connected to Firecrawl MCP server
Available tools: ['firecrawl_scrape', 'firecrawl_map', 'firecrawl_search', 'firecrawl_crawl', 'firecrawl_check_crawl_status', 'firecrawl_extract']


# Real-Time Research Agent Demo

This notebook demonstrates a LangChain v1 agent powered by vLLM and Firecrawl MCP tools for live web research.

In [ ]:
# Initialize vLLM on RunPod Pod
from langchain_openai import ChatOpenAI
from pydantic import SecretStr

if RUNPOD_ENDPOINT_URL == "YOUR_RUNPOD_ENDPOINT_HERE":
    raise ValueError(
        "RunPod endpoint not configured. Please:\n"
        "1. Deploy a RunPod pod with PyTorch template\n"
        "2. Install vLLM and start server with Hermes-2-Pro-Mistral-7B\n"
        "3. Add RUNPOD_ENDPOINT_URL and RUNPOD_API_KEY to .env file"
    )

# LangChain v1 / Modern OpenAI SDK parameters
# REDUCED TOKENS: Model has 4096 token context limit
llm = ChatOpenAI(
    api_key=SecretStr(RUNPOD_API_KEY),
    base_url=RUNPOD_ENDPOINT_URL,
    model="Qwen/Qwen3-8B",
    max_completion_tokens=2048,  # Balanced for tool calls + responses
    temperature=0.7
)

print("Connected to RunPod pod")

# Test the connection
try:
    response = llm.invoke("Say 'Hello from vLLM!' in one sentence.")
    print(f"Test response: {response.content}")
except Exception as e:
    print(f"Connection test failed: {e}")
    print("The model may still be loading. Wait a few minutes and try again.")

Connected to RunPod pod
Test response: Hello from vLLM! I am here to assist you with any questions or tasks you may have.


In [11]:
# Create the agent using LangChain v1

# Shortened system prompt to save tokens (4096 token context limit)
system_prompt = """You are a research assistant. Use firecrawl_search to find information and firecrawl_scrape for details. Be concise."""

# Create agent - pass only essential tools
agent = create_agent(
    model=llm,
    tools=firecrawl_tools,
    system_prompt=system_prompt
)

print("Agent created successfully!")
print(f"Tools available: {[tool.name for tool in firecrawl_tools]}")

# Quick test (MCP tools require async)
test_response = await agent.ainvoke({
    "messages": [{"role": "user", "content": "What can you help me research?"}]
})

print("\nAgent test response:")
print(test_response["messages"][-1].content)

Agent created successfully!
Tools available: ['firecrawl_scrape', 'firecrawl_map', 'firecrawl_search', 'firecrawl_crawl', 'firecrawl_check_crawl_status', 'firecrawl_extract']

Agent test response:
Please provide a specific topic or question you'd like me to research using firecrawl_search and firecrawl_scrape. For example, you could ask, "What are the top AI companies in 2023?" or "Find research papers on AI published in 2023."


In [12]:
# Demo 1: Live snapshot
response = await agent.ainvoke({
    "messages": [
        {"role": "user", "content": "Give me a quick snapshot of notable AI governance headlines from the past week in India. Highlight why they matter."}
    ]
})

print("Agent Response:")
print(response["messages"][-1].content)

Agent Response:
Here's a quick snapshot of notable AI governance headlines from the past week in India and why they matter:

1. **"India's AI Governance Guidelines: Their pro-innovation approach ...**" ([LiveMint](https://www.livemint.com/opinion/online-views/india-ai-governance-guidelines-pro-innovation-approach-template-global-south-artificial-intelligence-safety-rules/amp-11762768770780.html))
   - This article discusses how India's AI guidelines aim to balance technological advancement with societal safeguards, grounded in pro-innovation principles.

2. **"What govt's AI guidelines mean for tech regulation in India" ([The Indian Express](https://indianexpress.com/article/explained/explained-sci-tech/what-govts-ai-guidelines-mean-for-tech-regulation-10357983/))
   - The article explains that the recent guidelines suggest risk management within the existing framework of laws under the guiding principle of 'Do No Harm'.

3. **"PM launches Rs 1 Lakh Crore Research, Development and ...*